# Average polarimeter per resonance

```{autolink-concat}
```

In [ ]:
from __future__ import annotations

import logging
import os
from collections import defaultdict
from functools import lru_cache
from textwrap import dedent

import jax.numpy as jnp
import numpy as np
import plotly.express as px
import sympy as sp
import yaml
from ampform.sympy import PoolSum
from IPython.display import Latex
from tensorwaves.function import ParametrizedBackendFunction
from tqdm.auto import tqdm

from polarimetry import formulate_polarimetry
from polarimetry.amplitude import AmplitudeModel
from polarimetry.data import create_data_transformer, generate_phasespace_sample
from polarimetry.decay import Particle
from polarimetry.function import compute_sub_function
from polarimetry.io import (
    mute_jax_warnings,
    perform_cached_doit,
    perform_cached_lambdify,
)
from polarimetry.lhcb import (
    ParameterBootstrap,
    flip_production_coupling_signs,
    load_model_builder,
    load_model_parameters,
)
from polarimetry.lhcb.particle import load_particles

logging.getLogger("polarimetry.io").setLevel(logging.INFO)
mute_jax_warnings()
FUNCTION_CACHE: dict[sp.Expr, ParametrizedBackendFunction] = {}
MODEL_FILE = "../data/model-definitions.yaml"
PARTICLES = load_particles("../data/particle-definitions.yaml")

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)
    logging.getLogger("polarimetry.io").setLevel(logging.ERROR)

## Computations

In [ ]:
def formulate_all_models() -> dict[str, dict[int, AmplitudeModel]]:
    with open(MODEL_FILE) as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)
    allowed_model_titles = [title for title in data if "LS couplings" not in title]
    models = defaultdict(dict)
    for title in tqdm(
        allowed_model_titles,
        desc="Formulate models",
        disable=NO_TQDM,
    ):
        builder = load_model_builder(MODEL_FILE, PARTICLES, title)
        imported_parameters = load_model_parameters(MODEL_FILE, builder.decay, title)
        for reference_subsystem in [1, 2, 3]:
            model = builder.formulate(reference_subsystem)
            model.parameter_defaults.update(imported_parameters)
            models[title][reference_subsystem] = model
        models[title][2] = flip_production_coupling_signs(
            models[title][2], ["K", "L"]
        )
        models[title][3] = flip_production_coupling_signs(
            models[title][3], ["K", "D"]
        )
    return {i: {k: v for k, v in dct.items()} for i, dct in models.items()}


MODELS = formulate_all_models()
NOMINAL_MODEL_TITLE = "Default amplitude model"
NOMINAL_MODEL = MODELS[NOMINAL_MODEL_TITLE]
DECAY = NOMINAL_MODEL[1].decay

In [ ]:
def unfold_expressions() -> (
    tuple[
        dict[str, dict[int, sp.Expr]],
        dict[str, dict[int, sp.Expr]],
    ]
):
    intensity_exprs = defaultdict(dict)
    alpha_z_exprs = defaultdict(dict)
    for title, ref_models in tqdm(
        MODELS.items(),
        desc="Unfolding expressions",
        disable=NO_TQDM,
    ):
        for ref, model in tqdm(ref_models.items(), disable=NO_TQDM, leave=False):
            exprs = unfold_intensity_and_alpha_z(model, title, ref)
            intensity_exprs[title][ref] = exprs[0]
            alpha_z_exprs[title][ref] = exprs[1]
    return (
        {i: {k: v for k, v in dct.items()} for i, dct in intensity_exprs.items()},
        {i: {k: v for k, v in dct.items()} for i, dct in alpha_z_exprs.items()},
    )


def unfold_intensity_and_alpha_z(
    model: AmplitudeModel, title: str, reference_subsystem: int
) -> tuple[sp.Expr, sp.Expr]:
    builder = load_model_builder(MODEL_FILE, PARTICLES, model_id=title)
    _, _, alpha_z = formulate_polarimetry(builder, reference_subsystem)
    return (
        doit(doit(model.intensity).xreplace(model.amplitudes)),
        doit(doit(alpha_z).xreplace(model.amplitudes)),
    )


@lru_cache(maxsize=None)
def doit(expr: PoolSum) -> sp.Expr:
    return perform_cached_doit(expr)


ALPHA_Z_EXPRS, INTENSITY_EXPRS = unfold_expressions()

In [ ]:
def lambdify_expressions() -> (
    tuple[
        dict[str, dict[int, sp.Expr]],
        dict[str, dict[int, sp.Expr]],
        dict[str, dict[int, float]],
    ]
):
    intensity_funcs = defaultdict(dict)
    alpha_z_funcs = defaultdict(dict)
    original_parameters = defaultdict(dict)
    for title, ref_models in tqdm(
        MODELS.items(),
        desc="Lambdifying",
        disable=NO_TQDM,
    ):
        for ref, model in ref_models.items():
            intensity_expr = INTENSITY_EXPRS[title][ref]
            alpha_z_expr = ALPHA_Z_EXPRS[title][ref]
            intensity_funcs[title][ref] = cached_lambdify(intensity_expr, model)
            alpha_z_funcs[title][ref] = cached_lambdify(alpha_z_expr, model)
            str_parameters = {str(k): v for k, v in model.parameter_defaults.items()}
            original_parameters[title][ref] = str_parameters
    return (
        {i: {k: v for k, v in dct.items()} for i, dct in intensity_funcs.items()},
        {i: {k: v for k, v in dct.items()} for i, dct in alpha_z_funcs.items()},
        {
            i: {k: v for k, v in dct.items()}
            for i, dct in original_parameters.items()
        },
    )


def cached_lambdify(
    expr: sp.Expr, model: AmplitudeModel
) -> ParametrizedBackendFunction:
    func = FUNCTION_CACHE.get(expr)
    if func is None:
        func = perform_cached_lambdify(
            expr,
            parameters=model.parameter_defaults,
            backend="jax",
        )
        FUNCTION_CACHE[expr] = func
    str_parameters = {str(k): v for k, v in model.parameter_defaults.items()}
    func.update_parameters(str_parameters)
    return func


ALPHA_Z_FUNCS, INTENSITY_FUNCS, ORIGINAL_PARAMETERS = lambdify_expressions()

In [ ]:
N_EVENTS = 100_000
PHSP = generate_phasespace_sample(DECAY, N_EVENTS, seed=0)
for ref in tqdm([1, 2, 3], disable=NO_TQDM, leave=False):
    transformer = create_data_transformer(NOMINAL_MODEL[ref])
    PHSP.update(transformer(PHSP))
    del transformer

In [ ]:
def create_bootstraps() -> dict[int, ParameterBootstrap]:
    bootstraps = {
        ref: ParameterBootstrap(MODEL_FILE, DECAY, NOMINAL_MODEL_TITLE)
        for ref, model in NOMINAL_MODEL.items()
    }
    bootstraps[2] = flip_production_coupling_signs(bootstraps[2], ["K", "L"])
    bootstraps[3] = flip_production_coupling_signs(bootstraps[3], ["K", "D"])
    return {
        i: b.create_distribution(N_BOOTSTRAPS, seed=0) for i, b in bootstraps.items()
    }


def compute_statistics_weighted_alpha_z():
    weighted_alpha_z = defaultdict(list)
    resonances = get_resonance_to_reference()
    for resonance, ref in tqdm(
        resonances.items(),
        desc="Computing statistics",
        disable=NO_TQDM,
    ):
        _filter = [resonance.name.replace("(", R"\(").replace(")", R"\)")]
        intensity_func = INTENSITY_FUNCS[NOMINAL_MODEL_TITLE][ref]
        alpha_z_func = ALPHA_Z_FUNCS[NOMINAL_MODEL_TITLE][ref]
        original_parameters = ORIGINAL_PARAMETERS[NOMINAL_MODEL_TITLE][ref]
        for i in tqdm(range(N_BOOTSTRAPS), disable=NO_TQDM, leave=False):
            new_parameters = {k: v[i] for k, v in BOOTSTRAP_PARAMETERS[ref].items()}
            intensity_func.update_parameters(new_parameters)
            alpha_z_func.update_parameters(new_parameters)
            intensities = compute_sub_function(intensity_func, PHSP, _filter)
            alpha_z = compute_sub_function(alpha_z_func, PHSP, _filter).real
            alpha_z = compute_weighted_average(alpha_z, intensities)
            weighted_alpha_z[resonance.name].append(alpha_z)
            intensity_func.update_parameters(original_parameters)
            alpha_z_func.update_parameters(original_parameters)
    return {k: np.array(v) for k, v in weighted_alpha_z.items()}


def get_resonance_to_reference() -> dict[Particle, int]:
    subsystem_ids = dict(K=1, L=2, D=3)
    resonances = [
        c.resonance
        for c in sorted(
            DECAY.chains,
            key=lambda p: (subsystem_ids[p.resonance.name[0]], p.resonance.mass),
        )
    ]
    return {p: subsystem_ids[p.name[0]] for p in resonances}


def compute_weighted_average(v: jnp.ndarray, weights: jnp.ndarray) -> jnp.ndarray:
    return jnp.nansum(v * weights, axis=-1) / jnp.nansum(weights, axis=-1)


N_BOOTSTRAPS = 100
BOOTSTRAP_PARAMETERS = create_bootstraps()
STAT_WEIGHTED_ALPHA_Z = compute_statistics_weighted_alpha_z()

In [ ]:
def compute_systematic_weighted_alpha_z():
    weighted_alpha_z = defaultdict(list)
    resonances = get_resonance_to_reference()
    for resonance, ref in tqdm(
        resonances.items(),
        desc="Computing systematics",
        disable=NO_TQDM,
    ):
        _filter = [resonance.name.replace("(", R"\(").replace(")", R"\)")]
        for title in tqdm(MODELS, disable=NO_TQDM, leave=False):
            intensity_func = INTENSITY_FUNCS[title][ref]
            alpha_z_func = ALPHA_Z_FUNCS[title][ref]
            original_parameters = ORIGINAL_PARAMETERS[title][ref]
            intensity_func.update_parameters(original_parameters)
            alpha_z_func.update_parameters(original_parameters)
            intensities = compute_sub_function(intensity_func, PHSP, _filter)
            alpha_z = compute_sub_function(alpha_z_func, PHSP, _filter)
            alpha_z = compute_weighted_average(alpha_z, intensities).real
            weighted_alpha_z[resonance.name].append(alpha_z)
    return {k: np.array(v) for k, v in weighted_alpha_z.items()}


SYST_WEIGHTED_ALPHA_Z = compute_systematic_weighted_alpha_z()

## Result and comparison

LHCb values are taken from [from the original study](https://arxiv.org/pdf/2208.03262.pdf#page=23):

In [ ]:
def tabulate_alpha_z():
    lhcb_values = {
        "L(1405)": (-0.58, 0.05, 0.28, 0.01),
        "L(1520)": (-0.925, 0.025, 0.084, 0.005),
        "L(1600)": (-0.20, 0.06, 0.50, 0.03),
        "L(1670)": (-0.817, 0.042, 0.073, 0.006),
        "L(1690)": (-0.958, 0.020, 0.027, 0.006),
        "L(2000)": (+0.57, 0.03, 0.19, 0.01),
        "D(1232)": (-0.548, 0.014, 0.036, 0.004),
        "D(1600)": (+0.50, 0.05, 0.17, 0.01),
        "D(1700)": (-0.216, 0.036, 0.075, 0.011),
        "K(700)": (+0.06, 0.66, 0.24, 0.23),
        "K(1430)": (-0.34, 0.03, 0.14, 0.01),
    }
    model_ids = list(range(len(MODELS)))
    alignment = "r".join("" for _ in model_ids)
    header = " & ".join(Rf"\textbf{{{i}}}" for i in model_ids[1:])
    src = Rf"\begin{{array}}{{l|c|c|{alignment}}}" "\n"
    src += Rf" & \textbf{{this study}} & \textbf{{LHCb}} & {header} \\" "\n"
    src += R"\hline" "\n"
    src = dedent(src)
    for resonance in get_resonance_to_reference():
        src += f" {resonance.latex} & "
        stat_array = 1e3 * STAT_WEIGHTED_ALPHA_Z[resonance.name]
        syst_array = 1e3 * SYST_WEIGHTED_ALPHA_Z[resonance.name]
        syst_diff = syst_array[1:] - syst_array[0]
        value = syst_array[0]
        std = stat_array.std()
        min_ = syst_diff.min()
        max_ = syst_diff.max()
        src += Rf"{value:>+.0f} \pm {std:.0f}_{{{min_:+.0f}}}^{{{max_:+.0f}}}"
        src += " & "
        lhcb = lhcb_values.get(resonance.name)
        if lhcb is not None:
            val, stat, _, syst = lhcb
            src += Rf"{1e3*val:+.0f} \pm {1e3*stat:.0f} \pm {1e3*syst:.0f}"
        for diff in syst_diff:
            diff_str = f"{diff:>+.0f}"
            if diff == syst_diff.max():
                src += Rf" & \color{{red}}{{{diff_str}}}"
            elif diff == syst_diff.min():
                src += Rf" & \color{{blue}}{{{diff_str}}}"
            else:
                src += f" & {diff_str}"
        src += R" \\" "\n"
    src += R"\end{array}"
    return Latex(src)


tabulate_alpha_z()

## Distribution analysis

In [ ]:
layout_kwargs = dict(
    height=800,
    xaxis_title="Resonance",
    yaxis_title="ɑ̅<sub>z</sub>",
    font=dict(size=18),
)

fig = px.box(STAT_WEIGHTED_ALPHA_Z, points="all")
fig.update_layout(
    title="<b>Statistical</b> distribution of weighted ɑ̅<sub>z</sub>",
    **layout_kwargs,
)
fig.show()
fig.update_layout(font=dict(size=24), width=1200)
fig.write_image("_images/alpha-z-per-resonance-statistical.svg")

fig = px.box(SYST_WEIGHTED_ALPHA_Z, points="all")
fig.update_layout(
    title="<b>Systematics</b> distribution of weighted ɑ̅<sub>z</sub>",
    **layout_kwargs,
)
fig.show()
fig.update_layout(font=dict(size=24), width=1200)
fig.write_image("_images/alpha-z-per-resonance-systematics.svg")

:::{only} latex
![](_images/alpha-z-per-resonance-statistical.svg)
![](_images/alpha-z-per-resonance-systematics.svg)
:::